In [8]:
from nhldata import moneypuck
import pandas as pd

pd.set_option('display.max_columns', None) 

In [5]:
def get_shots_data(season):
    mp_conn = moneypuck.Connector()

    shots = mp_conn.shots_season(
        season = season
    )

    shots_df = pd.DataFrame(shots)

    return shots_df

In [9]:
df = get_shots_data(2024)

In [10]:
df.head(n = 5)

,shotID,arenaAdjustedShotDistance,arenaAdjustedXCord,arenaAdjustedXCordABS,arenaAdjustedYCord,arenaAdjustedYCordAbs,averageRestDifference,awayEmptyNet,awayPenalty1Length,awayPenalty1TimeLeft,awaySkatersOnIce,awayTeamCode,awayTeamGoals,defendingTeamAverageTimeOnIce,defendingTeamAverageTimeOnIceOfDefencemen,defendingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,defendingTeamAverageTimeOnIceOfForwards,defendingTeamAverageTimeOnIceOfForwardsSinceFaceoff,defendingTeamAverageTimeOnIceSinceFaceoff,defendingTeamDefencemenOnIce,defendingTeamForwardsOnIce,defendingTeamMaxTimeOnIce,defendingTeamMaxTimeOnIceOfDefencemen,defendingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMaxTimeOnIceOfForwards,defendingTeamMaxTimeOnIceOfForwardsSinceFaceoff,defendingTeamMaxTimeOnIceSinceFaceoff,defendingTeamMinTimeOnIce,defendingTeamMinTimeOnIceOfDefencemen,defendingTeamMinTimeOnIceOfDefencemenSinceFaceoff,defendingTeamMinTimeOnIceOfForwards,defendingTeamMinTimeOnIceOfForwardsSinceFaceoff,defendingTeamMinTimeOnIceSinceFaceoff,distanceFromLastEvent,event,gameOver,game_id,goal,goalieIdForShot,goalieNameForShot,homeEmptyNet,homePenalty1Length,homePenalty1TimeLeft,homeSkatersOnIce,homeTeamCode,homeTeamGoals,homeTeamScore,homeTeamWon,homeWinProbability,id,isHomeTeam,isPlayoffGame,lastEventCategory,lastEventShotAngle,lastEventShotDistance,lastEventTeam,lastEventxCord,lastEventxCord_adjusted,lastEventyCord,lastEventyCord_adjusted,location,offWing,penaltyLength,period,playerNumThatDidEvent,playerNumThatDidLastEvent,playerPositionThatDidEvent,playoffGame,roadTeamCode,roadTeamScore,season,shooterLeftRight,shooterName,shooterPlayerId,shooterTimeOnIce,shooterTimeOnIceSinceFaceoff,shootingTeamAverageTimeOnIce,shootingTeamAverageTimeOnIceOfDefencemen,shootingTeamAverageTimeOnIceOfDefencemenSinceFaceoff,shootingTeamAverageTimeOnIceOfForwards,shootingTeamAverageTimeOnIceOfForwardsSinceFaceoff,shootingTeamAverageTimeOnIceSinceFaceoff,shootingTeamDefencemenOnIce,shootingTeamForwardsOnIce,shootingTeamMaxTimeOnIce,shootingTeamMaxTimeOnIceOfDefencemen,shootingTeamMaxTimeOnIceOfDefencemenSinceFaceoff,shootingTeamMaxTimeOnIceOfForwards,shootingTeamMaxTimeOnIceOfForwardsSinceFaceoff,shootingTeamMaxTimeOnIceSinceFaceoff,shootingTeamMinTimeOnIce,shootingTeamMinTimeOnIceOfDefencemen,shootingTeamMinTimeOnIceOfDefencemenSinceFaceoff,shootingTeamMinTimeOnIceOfForwards,shootingTeamMinTimeOnIceOfForwardsSinceFaceoff,shootingTeamMinTimeOnIceSinceFaceoff,shotAngle,shotAngleAdjusted,shotAnglePlusRebound,shotAnglePlusReboundSpeed,shotAngleReboundRoyalRoad,shotDistance,shotGeneratedRebound,shotGoalProbability,shotGoalieFroze,shotOnEmptyNet,shotPlayContinued,shotPlayContinuedInZone,shotPlayContinuedOutsideZone,shotPlayStopped,shotRebound,shotRush,shotType,shotWasOnGoal,speedFromLastEvent,team,teamCode,time,timeBetweenEvents,timeDifferenceSinceChange,timeLeft,timeSinceFaceoff,timeSinceLastEvent,timeUntilNextEvent,wentToOT,wentToShootout,xCord,xCordAdjusted,xFroze,xGoal,xPlayContinuedInZone,xPlayContinuedOutsideZone,xPlayStopped,xRebound,xShotWasOnGoal,yCord,yCordAdjusted
0,0,52.0,57.0,57.0,-41.0,41.0,0.0,0,0,0,5,NJD,0,8.0,8.0,8.0,8.0,8.0,8.0,2,3,8,8,8,8,8,8,8,8,8,8,8,8,12.0,SHOT,,20001,0,8480045,Ukko-Pekka Luukkonen,0,0,0,5,BUF,0,,0,,7,0.0,0,HIT,0.0,0.0,HOME,69,69,-40,-40,HOMEZONE,0,,1,17,0,D,,,,2024,,Simon Nemec,8483495,8,8,8.0,8.0,8.0,8.0,8.0,8.0,2,3,8,8,8,8,8,8,8,8,8,8,8,8,-51.3401917459,51.3401917459,0.0,0.0,0,51.2249938995,0,,0,0,,0,1,0,0,0,WRIST,1.0,4.0,AWAY,NJD,8,,0,,8,3,11,,,57,57,0.238455,0.0125372,0.394229,0.301072,0.0228073,0.0309002,0.710867,-40,-40
1,1,33.0,71.0,71.0,-28.0,28.0,-6.0,0,0,0,5,NJD,0,16.0,18.5,18.5,14.3333333333,14.3333333333,16.0,2,3,29,29,29,29,29,29,6,8,8,6,6,6,148.677503342,MISS,,20001,0,8480045,Ukko-Pekka Luukkonen,0,0,0,5,BUF,0,,0,,9,0.0,0,GIVE,0.0,0.0,AWAY,-73,-73,9,9,HOMEZONE,1,,1,63,0,L,,,,2024,L,Jesper Bratt,8479407,7,7,10.0,14.5,14.5,7.0,7.0,10.0,2,3,29,29,29,8,8,29,0,0,0,6,6,0,-57.2647737279,57.2647737279,0.0,0.0,0,33.2866339542,0,,0,0,,0,1,0,0,0